In [ ]:
# Install dependencies as needed:
# pip install kagglehub[pandas-datasets]
import kagglehub

path = kagglehub.dataset_download("salvatoresaia/ev-charging-stations-us")

print("Path to dataset files:", path)

import os
files = os.listdir(path)
print("\nAvailable files:")
for file in files:
    print(f"  - {file}")

100%|██████████| 24.8M/24.8M [00:00<00:00, 33.5MB/s]


Extracting files...
Path to dataset files: /Users/anirudhannabathula/.cache/kagglehub/datasets/salvatoresaia/ev-charging-stations-us/versions/2

Available files:
  - EV_Charging_Stations_Feb82024.xlsx
  - EV_Charging_Stations_Jan312023.xlsx


In [ ]:
# Load the Excel files using pd.read_excel()
import pandas as pd

ev_charging_station_feb = os.path.join(path, 'EV_Charging_Stations_Feb82024.xlsx')
ev_charging_station_jan = os.path.join(path, 'EV_Charging_Stations_Jan312023.xlsx')

# Read the Excel files
df_feb = pd.read_excel(ev_charging_station_feb)
df_jan = pd.read_excel(ev_charging_station_jan)

print(f"February 2024 dataset shape: {df_feb.shape}")
print(f"January 2023 dataset shape: {df_jan.shape}")
print("\nFirst 5 records from February 2024:")
print(df_feb.head())


February 2024 dataset shape: (65134, 13)
January 2023 dataset shape: (54238, 13)

First 5 records from February 2024:
                        Station Name      Street Address         City State  \
0           LADWP - Truesdale Center  11797 Truesdale St   Sun Valley    CA   
1      Los Angeles Convention Center  1201 S Figueroa St  Los Angeles    CA   
2      LADWP - John Ferraro Building       111 N Hope St  Los Angeles    CA   
3         LADWP - Haynes Power Plant       6801 E 2nd St   Long Beach    CA   
4  LADWP - Harbor Generating Station    161 N Island Ave   Wilmington    CA   

     ZIP  EV Level1 EVSE Num  EV Level2 EVSE Num  EV DC Fast Count  \
0  91352                 NaN                57.0               2.0   
1  90015                 NaN                 7.0               NaN   
2  90012                 NaN               338.0              12.0   
3  90803                 NaN                19.0               1.0   
4  90744                 NaN                10.0         

In [8]:
df_feb.columns

Index(['Station Name', 'Street Address', 'City', 'State', 'ZIP',
       'EV Level1 EVSE Num', 'EV Level2 EVSE Num', 'EV DC Fast Count',
       'EV Network', 'EV Connector Types', 'Access Code', 'Access Detail Code',
       'Facility Type'],
      dtype='object')

In [9]:
df_jan.columns

Index(['Station Name', 'Street Address', 'City', 'State', 'ZIP',
       'EV Level1 EVSE Num', 'EV Level2 EVSE Num', 'EV DC Fast Count',
       'EV Network', 'EV Connector Types', 'Access Code', 'Access Detail Code',
       'Facility Type'],
      dtype='object')

In [10]:
df_feb

,Station Name,Street Address,City,State,ZIP,EV Level1 EVSE Num,EV Level2 EVSE Num,EV DC Fast Count,EV Network,EV Connector Types,Access Code,Access Detail Code,Facility Type
0,LADWP - Truesdale Center,11797 Truesdale St,Sun Valley,CA,91352,NaN,57.0,2.0,SHELL_RECHARGE,CHADEMO J1772 J1772COMBO,private,NaN,UTILITY
1,Los Angeles Convention Center,1201 S Figueroa St,Los Angeles,CA,90015,NaN,7.0,NaN,Non-Networked,J1772,public,NaN,PARKING_GARAGE
2,LADWP - John Ferraro Building,111 N Hope St,Los Angeles,CA,90012,NaN,338.0,12.0,Non-Networked,CHADEMO J1772 J1772COMBO,private,NaN,UTILITY
3,LADWP - Haynes Power Plant,6801 E 2nd St,Long Beach,CA,90803,NaN,19.0,1.0,Non-Networked,CHADEMO J1772 J1772COMBO,private,NaN,UTILITY
4,LADWP - Harbor Generating Station,161 N Island Ave,Wilmington,CA,90744,NaN,10.0,NaN,Non-Networked,J1772,private,NaN,UTILITY
...,...,...,...,...,...,...,...,...,...,...,...,...,...
65129,Davis Chevrolet of Delano,"505 Babcock Blvd,",Delano,MN,55328,NaN,NaN,1.0,EV Connect,J1772COMBO,public,NaN,NaN
65130,9000 N Division St,9000 N Division St,Spokane,WA,99218,NaN,1.0,6.0,EV Connect,J1772 J1772COMBO,public,NaN,NaN
65131,Truist Park Purple Deck,TBD,Atlanta,GA,30339,NaN,4.0,NaN,FLASH,J1772,public,NaN,PUBLIC
65132,One Victory Park Garage,2323 Victory Ave.,Dallas,TX,75219,NaN,5.0,NaN,FLASH,J1772,public,NaN,WORKPLACE


In [12]:
df_feb[df_feb['Access Code'] == 'public']
# These are .xlsx files, so we need to use read_excel()

,Station Name,Street Address,City,State,ZIP,EV Level1 EVSE Num,EV Level2 EVSE Num,EV DC Fast Count,EV Network,EV Connector Types,Access Code,Access Detail Code,Facility Type
1,Los Angeles Convention Center,1201 S Figueroa St,Los Angeles,CA,90015,NaN,7.0,NaN,Non-Networked,J1772,public,NaN,PARKING_GARAGE
8,California Air Resources Board,9530 Telstar Ave,El Monte,CA,91731,NaN,3.0,NaN,Non-Networked,J1772,public,NaN,STATE_GOV
10,Scripps Green Hospital,10666 N Torrey Pines Rd,La Jolla,CA,92037,NaN,1.0,NaN,Non-Networked,J1772,public,NaN,HOSPITAL
11,Galpin Motors,15421 Roscoe Blvd,Sepulveda,CA,91343,NaN,2.0,NaN,Non-Networked,J1772,public,CALL,CAR_DEALER
12,Galleria at Tyler,1299 Galleria at Tyler,Riverside,CA,92503,NaN,4.0,NaN,Non-Networked,J1772,public,NaN,SHOPPING_MALL
...,...,...,...,...,...,...,...,...,...,...,...,...,...
65129,Davis Chevrolet of Delano,"505 Babcock Blvd,",Delano,MN,55328,NaN,NaN,1.0,EV Connect,J1772COMBO,public,NaN,NaN
65130,9000 N Division St,9000 N Division St,Spokane,WA,99218,NaN,1.0,6.0,EV Connect,J1772 J1772COMBO,public,NaN,NaN
65131,Truist Park Purple Deck,TBD,Atlanta,GA,30339,NaN,4.0,NaN,FLASH,J1772,public,NaN,PUBLIC
65132,One Victory Park Garage,2323 Victory Ave.,Dallas,TX,75219,NaN,5.0,NaN,FLASH,J1772,public,NaN,WORKPLACE
